<img src="Images/IMG-dirty-data.jpg">
When processing data with machine learning methods, one is often confronted with data sets that consist of many different features. These features are often heterogeneous: some are coded by a continuous numerical value, some can only assume certain discrete levels, some may be coded by a word or even whole sentences. Often individual values of features are also missing, for example, because a sensor in a multimodal measuring system failed for a short time or because not all questions in a questionnaire were answered. However, most machine learning methods expect fully occupied, exclusively numerical feature vectors at the input.

<img style="float: right; margin:5px 0px 0px 10px" src="Images/IMG-lego-logo.svg" alt="Lego-Logo" height="128" width="128">

In this notebook you will learn and apply procedures that can be used in the preprocessing of a mixed data set ("dirty data"). As an example data set we use a file that was created by extracting the data of all products available on www.lego.com (see also [Web Scraping](https://en.wikipedia.org/wiki/Web_scraping)). The content of each column should be largely self-explanatory based on the column title.

## Content
<table style="width:256; border: 1px solid black; display: inline-block">
  <tr>
    <td  style="text-align:right" width=64px><img src="Images/IMG-csv-in.png" style="float:left"></td>
      <td style="text-align:left" width=128px>
          <a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#import_data'>Import data</a>
      </td>
  </tr>
  <tr>
    <td style="text-align:right"><img src="Images/IMG-magnifying-glass.png" style="float:left"></td>
    <td style="text-align:left" width=128px><a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#analyze_data'>Analyze data</a>
      </td>
  </tr>
    <tr>
    <td style="text-align:right"><img src="Images/IMG-broom.png" style="float:left"></td>
    <td style="text-align:left" width=128px><a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#clean_data'>Clean data</a>
        </td>
    </tr>
    <tr>
    <td style="text-align:right"><img src="Images/IMG-csv-out.png" style="float:left"></td>
    <td style="text-align:left" width=128px><a style="color:black; font-size:14px; font-weight:bold; text-decoration:none" href='#save_data'>Save data</a>
        </td>
  </tr>
</table>


<a id='import_data'></a><div><img src="Images/IMG-csv-in.png" style="float:left"> <h2 style="position: relative; top: 6px">1. Import data</h2>
<p style="position: relative; top: 10px">
The unprocessed raw data is available in the format of a <a href="https://de.wikipedia.org/wiki/CSV_(Dateiformat)">CSV data</a>. Most <a href="https://de.wikipedia.org/wiki/H%C3%B6here_Programmiersprache">high-level programming languages</a> (such as Python or Matlab) already have methods for importing these simply structured files. In the case of Python this goes e.g. with the function <a href="https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv">read_csv()</a> from the module <a href="https://pandas.pydata.org/docs/getting_started/10min.html#min">pandas (click here for a 10-minute tour) </a>.
</p>

It works like this, for example:

```python
import pandas as pd
data = pd.read_csv("Path/to/data.csv")
```
Try it out in the next cell! The CSV file ``lego_sets.csv`` is located in the ``Data`` subdirectory.

In [9]:
# Here you can write Python code and execute it with Shift+Enter!

# Import model

# Read CSV files (save data as DataFrame)


Let's habe a look at how pandas saves the data it has read. You can of course find this information in the documentation, but you can also use the function [Built-In Funktion type()](https://docs.python.org/3/library/functions.html#type):

In [ ]:
# What type is the data structure created?


As you should now know, the data of the Lego set is saved as [``DataFrame``](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame).

A ``DataFrame`` is a tabular data structure in pandas that contains an ordered set of columns. Each column can have a different data type (numeric, string, Boolean, etc.). A ``DataFrame`` has both a row index and a column index.

To get a quick look at the data structure, pandas offers the functions [``head()``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [``tail()``](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html#pandas.DataFrame.tail) to display the first or last lines of the structure.


In [ ]:
## Take a look at the data structure


Of course, you can also specifically index individual elements from the ``DataFrame``. This works for whole columns similar to the Python standard type [*dictionary*](https://docs.python.org/3/tutorial/datastructures.html#dictionaries):

```python
some_dataFrame['some_column_name']
```

If you want to return multiple columns, you can pass a list of column names to the ``[]`` operator:

```python
some_dataFrame[['some_column_name', 'another_column_name']]
```

In [ ]:
# Output all product IDs

# Output only the rating stars and the number of reviews


If you (additionally) want to limit the selection to individual lines, pandas offers two options ``.loc`` and ``.iloc``. ``.loc`` is intended for use with labels and ``.iloc`` for purely numerical indices:

```python
some_dataFrame.loc[20:40, ['some_column_name', 'another_column_name]]
```

```python
some_dataFrame.iloc[20:40, 2:3]
```

The variant with ``.loc`` is very useful for selecting _individual_ rows or columns, while ``.iloc`` is more suitable for selecting _spans_ of rows or columns.

__Attention__: The two variants have different inclusiveness of the index ranges!

In [ ]:
# Return 100 rows from any two columns anywhere in the data set. Compare .loc and .iloc!

More about the object ``DataFrame`` can be found at [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html).

---

<a id="analyze_data"></a><div><img src="Images/IMG-magnifying-glass.png" style="float:left"> <h2 style="position: relative; top: 6px">2. Analyze data</h2>

<p style="position: relative; top: 10px">
After the data has been successfully read and we have informed ourselves about the structure of the data, we can deal with the actual content. First of all, we are interested in the form in which the data for each column is stored. A pandas <code>DataFrame</code> offers the property <code>.dtypes</code>:
</p>

In [ ]:
# Output the value of the ".dtypes" attribute of your data set


You should notice that some of the columns are numeric (``float64``) while others are of type ``object``. Since our goal is to create a purely numeric data set, we have to replace or convert the non-numeric features in a suitable way. In order to keep an overview, we therefore create a list with all the names of the columns to be converted.

Here another type of indexing of ``DataFrame`` can help us: indexing with a boolean array. We create an array of boolean values, whose length is equal to the number of dimensions to be indexed. Here ``True`` stands for taking the index into account in the call and ``False`` for suppressing the index.

```python
some_dataFrame_with_three_columns.loc[:,[True, True, False]] # Gibt die ersten zwei Spalten des dreispaltigen DataFrame zurück
```

You can also calculate this array directly in the call for well readable code, e.g. through relational operators (``<>==``).

Try it out right away. First read the attribute of the ``DataFrame`` with the column titles (``.columns``). Then only save the titles for which the attribute ``.dtypes`` has the value ``'object'`` as a list (conversion with ``list()``).


In [ ]:
# Reading the column titles


In addition to unsuitable types, missing entries in the data set can also prevent subsequent processing. So we are next interested in whether and where entries are missing. Pandas provides the method ``isna()``, which returns ``True`` for missing entries and ``False`` for existing entries. With the method ``any()`` you can then check whether a value is missing somewhere in a column:

```python
some_dataFrame.isna().any()
```

In [ ]:
# Find the columns that are missing values.


If only a few values are missing in a column, they may be useful for padding (see <a href="#clean_data">Clean up data</a>). However, if a lot of data is missing, the whole column may have to be removed. With the method ``count()``, all existing entries in each column can be counted.

```python
some_dataFrame.count()
```

In [ ]:
# Return the number of entries in each column


Now put the numbers in relation to the total number of entries. To do this, you can first create a [Tupel](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences) with the number of rows and columns with the ``shape`` attribute.

In [ ]:
# Return the relative proportion of the existing entries in each column


Even the column with the most missing entries still has more than 80% of existing entries, which makes it seem reasonable to fill in the values ([Imputation](https://de.wikipedia.org/wiki/Imputation_(Statistik))). However, this decision has to be weighed up anew for each concrete problem and no generally applicable guide values can be given here.

For our purposes, however, we now assume that there is enough data in each column. In the next section we will now convert the non-numeric columns identified above and fill in the missing values.

---

<a id="clean_data"></a><div><img src="Images/IMG-broom.png" style="float:left"> <h2 style="position: relative; top: 6px">3. Clean data</h2>

As already described above, most machine learning methods require complete numerical input data. Therefore we start the data processing first with the conversion of all non-numeric columns. We will then fill in the missing entries identified above in the most skilful way possible.

__Attention:__ Of course, filling in missing entries is an intervention and you __thereby falsify the data!__ Nevertheless, this can be a legitimate approach, if not too many elements are missing. A fixed size, what counts as too much, cannot be specified here. Basically, however, you should always only ever validate and test your __models trained on filled data only on complete data!__

<h3> 3.1 Non-numeric features</h3>

As stated above, we need to convert the data of some columns into numeric values. There is no general recipe for this problem, but it must be considered in each individual case which conversion makes sense. Therefore, in the following we will work on each column individually.

<h4>3.1.1 Age</h4>

The column ``ages`` has the ``dtype`` ``'object'``. This is surprising at first, after all, the age should be easy to code numerically. So let's take a look at the first few entries of the column again:

In [ ]:
# Output the head of the 'ages' column


You will certainly notice that Lego specifies the age as intervals, as is customary in the industry. The data set apparently uses strings of different formats, since two different ones can be seen in the head. How many unique values there are can be returned using the ``unique()`` method:

```python
some_dataFrame.unique()
```

In [ ]:
# Output the possible values of the column 'ages'


We see that the format of the age specification contains either a minimum _and_ a maximum age or _only_ a minimum age. It is therefore advisable to replace the column ``'ages'`` with two new columns ``'age_min'`` and ``'age_max'``.

However, it is still problematic that the maximum age is missing for some data. But we can fill in the global maximum value of ``99`` (also typical for the industry) here. So we're actually already anticipating and doing our first imputation: we replace a missing value with a statistically found value: the maximum.

Another obstacle to further processing is the notation ``'½'``. Therefore we want to replace this character as well.  __Attention:__ ``½`` is not the same string as ``1/2``.

So we want to replace one character with another in every string in a column. This is very easy in pandas:

```python
some_dataFrame['column_label'] = some_dataFrame['column_label'].str.replace('old_string', 'new_string')
```

In [ ]:
# Replace the character string '+' in the column 'ages' with the character string '-99'

# Also replace the string '½' in the column 'ages' with the string '.5'


Now all strings have the same format and we can create the new columns ``'age_min'`` and ``'age_max'`` by splitting the strings into ``'ages'`` at the character ``-``. To separate a character string at a specific separator, pandas provides ``.str.split()``.

```python
split_string = some_dataFrame['column_label'].str.split('string_to_split_on', expand=True)
```

__Note:__ If the ``expand`` argument is not set to``True``, the method returns the substrings found as a series of tuples, which is not ideal for further processing here.

In [ ]:
# Separate the values in 'ages' with the hyphen '-',

# and save the first element in the column 'age_min'

# and the second element in the column 'age_max'


Now that we have processed all of the information from the ``'ages'`` column, we can delete the column from the data set. We can use the ``drop()`` method for this. With the argument ``inplace=True`` we delete the column directly in the data set and not only return a copy of the data set without the column.

```python
some_dataFrame.drop(columns=['column_to_drop'], inplace=True)
```

In [ ]:
# Delete the redundant column 'ages' from the data set


Finally, we need to convert the new columns to numeric data types. Note that although we already have numerical values in the columns, these are still represented as strings. We can easily do a type conversion using ``astype()``.

```python
some_dataFrame['column_of_some_dtype'].astype('new_dtype')
```

Please refer to the documentation of method [astype()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) for the exact procedure of type conversion.

In [ ]:
# Convert the entries of the columns 'age_min' and 'age_max' to the data type 'float32'

#### 3.1.2 Level of difficulty

The level of difficulty is stored in the ``'review_difficulty'`` column. View the unique entries in this column (see above).

In [ ]:
# Show unique entries in 'review_difficulty'


We find that although the degree of difficulty is not directly numerically coded, it is based on a five-level [Ordinal scale](https://de.wikipedia.org/wiki/Ordinalskala). In addition, some entries seem to be missing, as there is also ``nan`` in the list.

The level of difficulty could therefore also be mapped on a scale from 1 to 5, whereby 'Very Easy' should be mapped to 1 and 'Very Challenging' to 5, so that 'Very Easy' < 'Very Challenging' applies.

We can easily do this with the help of ``map()``, which only exists for objects of the type ```Series```. However, each column of a ```DataFrame``` object is also a ```Series``` object. So the following goes:

````python
some_dataFrame['column_name'] = some_dataFrame['column_name'].map({'key1': 'value1', 'key2': 'value2'}, na_action='ignore')
````
__Note:__ Since we have not yet filled in the missing values (``na``), the argument ``na_action='ignore'`` should be set, so that the ``na`` values are simply passed through.

In [ ]:
# Determine the order of the difficulty levels

# Map strings to the numerical values


The missing entries could already be filled in now.  For example, all missing entries could be assumed to be of the ``'Average'`` difficulty level. Or you could use the actual numerical average or the most common value. However, it is likely that the level of difficulty also depends on other columns such as the age or the number of parts. Therefore, it makes sense to insert the missing entries later.

#### 3.1.3 Country code

Look at the values that appear in the ``'country'`` column.

In [ ]:
# Show unique entries in 'country'


Similar to the difficulty level, we only represented a limited number of values here. However, no meaningful numerical order can be determined for the country codes. Instead, you can use so-called [Dummy variables](https://de.wikipedia.org/wiki/Dummy-Variable) to signal affiliation to the respective country in binary.

Pandas offers the function ``get_dummies()`` for this purpose, which makes coding extremely easy:

```python
dummy_variables = some_dataFrame['column_name'].str.get_dummies()
```

__Note:__ The dummy variables can then be appended to the data set with the help of [concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html). Make sure to concatenate the _columns_ together!

In [ ]:
# Calculate dummy variables

# Append to data set

# Delete the 'country' column


#### 3.1.4 Name of the set

The name of the set is a particular challenge as almost every set has its own name. Coding this variable with dummy variables would therefore increase the number of variables enormously, which is very unfavorable for most models that are supposed to work with this data ([Curse of Dimensionality](https://de.wikipedia.org/wiki/Fluch_der_Dimensionalit%C3%A4t)). Therefore, this column should be coded as an unordered, categorical variable. For this purpose there is the data type ``category`` in pandas, to which we can easily convert:

```python
categorical_column = some_dataFrame['non_categorical_column'].astype('category')
```

The categorical variable can then in turn be converted into a numeric type by issuing the code assigned to each level of the variable:

```python
numerical_categorical_column = categorical_column.cat.codes
```

__Attention:__ This coding implies a ranking of the categories, but it is in fact not justifiable.

In [ ]:
# Convert the column 'set_name' into a categorical variable

# Assign the codes of the categories of the variable to the 'set_name' column


#### 3.1.4 Name of the topic
There are also many different unique values for the column ``'theme_name'``, although not quite as many as in the case of ``'set_name'``. Coding with dummy variables can already fatally increase the feature space here. In fact, in such a borderline case, one should compare the performance of the model to be trained with dummy coding and categorical coding to make a decision. Since this is not possible for us here, we simply assume that the dummy coding does not yet allow the feature space to become too high-dimensional.

In [ ]:
# Perform the dummy coding for 'theme_name' as described above


#### 3.1.5 Product descriptions
The product descriptions ``prod_desc`` and ``prod_long_desc`` are available as continuous text and therefore represent a particular challenge. Let's take a look at some of the descriptions first. It is recommended not to select the first or last few entries, but to make a random selection. Pandas offers the method ``.sample()``:

```python
some_dataFrame.sample(number_of_samples)
```

In [ ]:
# Output 10 random entries of the column 'prod_desc'


It seems that the short product description does not contain any systematic information. At this point, one could pursue many different approaches and, for example, analyse the verbs and adjectives used (using [Natural Language Processing](https://de.wikipedia.org/wiki/Computerlinguistik) methods). For the scope of this notebook, however, this leads too far and instead we limit ourselves to the number of words in the short description. To do this, we first have to split the strings at the spaces (see above) and count the number of separated units.

To determine the number of elements in an object Python offers the built-in function [``len()``](https://docs.python.org/3/library/functions.html#len).

In [ ]:
# Split the strings in the 'prod_desc' column at the space

# Determine the number of words (attention: loop or better list comprehension and treatment of NaN (=0 words) necessary!)

# Replace the column 'prod_desc' with the determined number


Now let's take a look at the detailed description:

In [ ]:
# Output 10 random examples from the column 'prod_long_desc' (attention: a loop and print() is needed here to see the whole description)


Elaborate analyzes could of course also be used here. However, we also find that many descriptions include the physical dimensions of the Lego sets. These could easily be numerically coded if we can extract them from the continuous text. For the sake of simplicity, we will only consider the first dimension given in the description for each dimension.

To find an occurring pattern in a string (such as ``<NUMBER><UNIT><SPACE><DIMENSION>``), [regular expressions](https://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck) are suitable. Regular expressions are a very powerful concept that is used in many programming languages and frameworks. With a regular expression, even complex patterns can be formulated in an abstract way, so that they can be used for comparison. A detailed digression on regular expressions leads too far at this point, which is why we have already given the expressions for the patterns to be found. After defining a pattern, you can use the pandas method ``extract()`` to extract the first occurrence of the pattern:

```python
some_dataFrame.str.extract(regularExpression)
```
__Attention:__ Since there are many special characters in regular expressions, they are usually defined in Python as [Raw String](https://www.journaldev.com/23598/python-raw-string) (with a preceding ``r``).


In [ ]:
# Use str.extract() and the string r'(?:(?P<high>\d+)(?:” high))' to extract the first height in inches

# You can find the width with r'(?:(?P<wide>\d+)(?:” wide))'

# You can find the length with r'(?:(?P<long>\d+)(?:” long))'

# Add the dimensions as new columns 'height', 'width', 'length' in the DataFrame lego_data



# Remove the column 'prod_long_desc'

### 3.2 Missing values

The data set now only contains numerical features. However, not all entries are available in every column. However, many algorithms expect complete data sets. The easiest way to deal with missing values is to simply delete the relevant row or column. But this is of course associated with loss of information and should therefore be avoided. So in this section we will use different methods to fill in the missing values.

Let's look again at how many values are missing in each column (see above).

In [ ]:
# Output the number of missing values per column


We note that the ratings (reviews and star ratings) in particular are incomplete. The aim of filling in is to complete the data set without distorting the distribution within the respective column. It therefore makes sense to first look at the respective distribution before deciding the further procedure.

Pandas offers the ```.hist()``` method for this purpose, with which histograms of the columns (or a selection of columns) of a data frame can be displayed.

```python
some_dataFrame[list_of_columns_of_interest].hist()
```

In [ ]:
# Output histograms of the columns with missing values


We find that most of the columns are not normally distributed. It would therefore be unfavorable to fill in the missing values here with the mean value of the column. Instead, we can fill in the most frequent value for such skewed distributions.

For the determination of the most frequent value and the mean value, pandas offers the methods ```.mean()``` and ```.mode()```. For filling in missing values there is the method ```.fillna()```.

```python
some_dataFrame[columns].mean() # Gibt Mittelwerte der Spalten <columns> zurück
some_dataFrame[columns].mode() # Gibt häufigste Werte der Spalten <columns> zurück ACHTUNG: Rückgabewert ist nicht skalar!
some_dataFrame[columns].fillna(value) # Füllt alle fehlenden Werte in den Spalten <columns> mit <value> auf
```



In [ ]:
# Fill in the column (s) with (approximately) normally distributed values with the mean

# Fill in the column (s) with non-normally distributed values with the most frequent value


Here also different, problem-adapted options for filling are often useful. For example, one could try to predict the degree of difficulty from the number of parts, and thus make the filling in the missing values a regression problem of its own. Or you try to find a line that is as similar as possible to a line in which a value is missing and accept the missing value (nearest neighbor method). The popular Python toolkit [_scikit-learn_](https://scikit-learn.org/stable/) offers, for example, a whole range of possibilities for imputation in the module [```sklearn.impute```](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute). However, this will not be discussed further in the context of this notebook.

<a id="save_data"></a><div><img src="Images/IMG-csv-out.png" style="float:left"> <h2 style="position: relative; top: 6px">4. Export data</h2>

<p style="position: relative; top: 10px">
Thus the data set is now complete and consists only of numerical characteristics. It only has to be exported to a CSV file and can then be used by models for regression or classification without further processing.

Pandas also offers a simple method for this: ```.to_csv()```

```python
some_dataFrame.to_csv(filename)
```
</p>


In [ ]:
# Export the cleaned up data set to a file named 'tidy_lego_sets.csv' in the subfolder 'Data'


<p style="text-align:center; font-weight:bold">Congratulations! You have successfully sorted the messy pile of Lego.</p>

<img src="Images/IMG-lego-messy-to-tidy.png" width="1080px" style="float:center"/>


<a style="float:left;background-color:black;color:white;text-decoration:none;padding:4px 6px;font-family:-apple-system, BlinkMacSystemFont, &quot;San Francisco&quot;, &quot;Helvetica Neue&quot;, Helvetica, Ubuntu, Roboto, Noto, &quot;Segoe UI&quot;, Arial, sans-serif;font-size:12px;font-weight:bold;line-height:1.2;display:inline-block;border-radius:3px" href="https://unsplash.com/@egnaro?utm_medium=referral&amp;utm_campaign=photographer-credit&amp;utm_content=creditBadge" target="_blank" rel="noopener noreferrer" title="Download free do whatever you want high-resolution photos from Rick Mason"><span style="display:inline-block;padding:2px 3px"><svg xmlns="http://www.w3.org/2000/svg" style="height:12px;width:auto;position:relative;vertical-align:middle;top:-2px;fill:white" viewBox="0 0 32 32"><title>unsplash-logo</title><path d="M10 9V0h12v9H10zm12 5h10v18H0V14h10v9h12v-9z"></path></svg></span><span style="display:inline-block;padding:2px 3px">Photo by Rick Mason</span></a><a style="float:right;background-color:black;color:white;text-decoration:none;padding:4px 6px;font-family:-apple-system, BlinkMacSystemFont, &quot;San Francisco&quot;, &quot;Helvetica Neue&quot;, Helvetica, Ubuntu, Roboto, Noto, &quot;Segoe UI&quot;, Arial, sans-serif;font-size:12px;font-weight:bold;line-height:1.2;display:inline-block;border-radius:3px" href="https://www.reddit.com/r/lego/comments/7mhq2b/finished_kind_of_organizing_my_spare_lego_pieces/" target="_blank" rel="noopener noreferrer" title="Download free do whatever you want high-resolution photos from Rick Mason"><span style="display:inline-block;padding:2px 3px"><svg xmlns="http://www.w3.org/2000/svg" style="height:12px;width:auto;position:relative;vertical-align:middle;top:-2px;fill:white" viewBox="0 0 32 32"><title>unsplash-logo</title><path d="M10 9V0h12v9H10zm12 5h10v18H0V14h10v9h12v-9z"></path></svg></span><span style="display:inline-block;padding:2px 3px">Photo by rexorhun</span></a>


<div>Icons made by <a href="https://www.flaticon.com/authors/swifticons" title="Swifticons">Swifticons</a> from <a href="https://www.flaticon.com/" title="Flaticon">www.flaticon.com</a></div>
<div>Notebook erstelt von Yifei Li & <a href="mailto:simon.stone@tu-dresden.de?Subject=Frage%20zu%20Jupyter%20Notebook%20Lego%20Sets" target="_top">Simon Stone</a></div>